## $H \to \gamma \gamma$ Fit

This is an example of fitting the 2-photon invariant mass to determine the number of Higgs signal events.

We have a data set consists of ~ 30000 di-photon invariant mass values from a tetx file (**Hgg.txt**) which should be in the current directory. 

We perform a binned maximum likelihood fit (for reducing the CPU time) using RooFit.  

### 1. Reading Input data set

In [ ]:
TTree tree("tree","tree");
int nevt = tree.ReadFile("Hgg.txt","x");
if (nevt <= 0) {
    Error("fitHgg","Error reading data from input file ");
}
std::cout << "Read " << nevt << " from the file " << std::endl;

### 2.  Create of histogram representing the  data

We make an histogram with 100 bins from 110 to 160 where we fill the invariant mass data from the tree

In [ ]:
auto h1 = new TH1D("h1","Invariant Mass distribution;Mass;",100,110,160);
tree.Draw("x >> h1");
h1->Draw(); gPad->Draw();

### 3. Model creation

We make now the model using the capabilities of TF1 using the NSUM operator (normalized sum of functions). 

We assume a Gaussian distribution for the signal and a double  exponential distribution for the background as following: 

$$ P(x | \mu , \nu ) = n_{sig} \times G(x | M , \sigma) + n_{bkg} \times E(x|a_1,a_2)$$

where $G (x | M , \sigma)$ is the Gaussian distribution for the signal and $E(x|a_1,a_2)$ is the exponential distribution describing the background. 

$$E(x|a_1,a_2) = \frac{ e^{( - a1 * x/100 - a2 * (x/100)^2 )}}{\int e^{-(.....)} dx }$$ 

In [ ]:
auto fsig = new TF1("fsig","[Constant]*TMath::Gaus(x,[Mass],[Sigma])");

In [ ]:
auto fbkg = new TF1("fbkg","[Constant]*exp(-([a1]*x)/100.-[a2]*(x/100)*(x/100))");

In [ ]:
auto fmodel = new TF1("model","NSUM(fsig,fbkg)",110,160);

### 4. Fit the data 

We do first a fit to the histogram using the bacground only function to get reasanable values for the parameters

In [ ]:
fbkg->SetParameter("Constant",1.E7);
fbkg->SetParameter("a1",8);
fbkg->SetParameter("a2",2);
fbkg->SetRange(110,160); fbkg->Draw();
ROOT::Math::MinimizerOptions::SetDefaultMinimizer("Minuit2");
h1->Fit(fbkg,"L");
gPad->Draw();

#### Full data fit using the created model (Higgs signal + background)

We first set initial parameter in the function

In [ ]:
fmodel->Print("V")

In [ ]:
fmodel->SetParameter("Coeff0",100);
fmodel->SetParameter("Coeff1",10000);
fmodel->SetParameter("Mass",124);
fmodel->SetParameter("Sigma",1);
fmodel->SetParameter("a1",fbkg->GetParameter("a1"));
fmodel->SetParameter("a2",fbkg->GetParameter("a2"));

Now we fit the histogram. We perform a Binned likelihood fit (option L)

In [ ]:
res = h1->Fit(fmodel,"L S");
fmodel->Draw();
h1->Draw();
gPad->Draw();

The number of signal/background events are equal to the Coefficients in the Normalized sum function divided by the bin width

In [ ]:
double bw = h1->GetBinWidth(1);
std::cout << "Number of Higgs events = " << res->Parameter(0)/bw << " +/- " << res->ParError(0)/bw << std::endl;
std::cout << "Number of Backg. events = " << res->Parameter(1)/bw << " +/- " << res->ParError(1)/bw << std::endl;

### 4. Compute the Significance

For computing the significance we look at the Delta of the Likelihood obtained by fitting fixing the number of signal events to be zero and the full fit. 
The significance is equal to $\sqrt {\Delta logL}$.

We do then a background only fit 

In [ ]:
auto fmodel2 = new TF1(*fmodel); 
fmodel2->FixParameter(0,0);
fmodel2->FixParameter(2, res->Parameter(2));
fmodel2->FixParameter(3, res->Parameter(3));
fmodel2->SetLineColor(kBlue); 

In [ ]:
res2 = h1->Fit(fmodel2,"L S +");
gPad->Draw();

In [ ]:
std::cout << "Significance is = " << sqrt( res2->MinFcnValue() - res->MinFcnValue() ) << std::endl;